# Lecture 8: Model Fitting


One of the most common things in scientific computing is model fitting. Numerical Recipes devotes a number of chapters to this.

* Least squares fitting versus chisquare fitting
* scipy "curve_fit"
* Making out own fitter

Later, we will also need to be able to read data, since we will try fitting real data and interpreting the results. We will be using an ascii file reader to parse real astronomical data. 

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import math

from scipy import signal

In [ ]:
def gauss(x, amplitude=1, mu=0, sig=1):
    return amplitude * np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))

In [ ]:
noise_width = 10

n_points = 64

time_axis = np.linspace(0,16,n_points,endpoint=False)
fine_time_axis = np.linspace(time_axis[0], time_axis[-1], 10 * n_points, endpoint=False)

In [ ]:
amplitude = 120
mean = 6.5
width = 0.7

noise = np.random.normal(0, noise_width, len(time_axis))
pedestal = -30
source = -1 * gauss(time_axis, amplitude, mean, width)

In [ ]:
contaminated_pulse = source + noise + pedestal
errors = np.ones(len(contaminated_pulse)) * 10

for i in contaminated_pulse:
    if i > 0:
        i = -1 #setting to -1 and not to zero so as to not break some math later.
#this line basically just checks that all of our data points are negative 
#it's basically saying "our electronics can only measure signals in one direction"


plt.errorbar(time_axis, contaminated_pulse, errors,fmt='o')
plt.xlabel("t (ns)")
plt.ylabel("v (mV)")

In [ ]:
#Passing arguments this way allows you to pass an arbitrarily large number of arguments to a function. 
#for example: 

def foo(x, *args):
    print("args:")
    for arg in args:
        print(arg)
        
def foo2(x, **kwargs):
    print("kwargs:")
    for key in kwargs.keys():
        print("{0:s}: {1}".format(key, kwargs[key]))
    
    
def foo3(x, *args, **kwargs):
    print("args:")
    for arg in args:
        print(arg)
    print("-----")    
    print("kwargs:")
    for key in kwargs.keys():
        print("{0:s}: {1}".format(key, kwargs[key]))
        

foo(time_axis, 10, 19, "stealth")       
print()
foo2(time_axis, a=10, b=12, name="stealth_bird")
print()
foo3(time_axis, 10, 19, "jupiter", r="one_string", g='a', something_else="some other string", input_list=[12,13,190])

In [ ]:
def gauss(x, *args):

    A, mu, sigma = args
    return A*np.exp(-(x-mu)**2/(2.*sigma**2)) 

In [ ]:
#curve_fit is a powerful and commonly used fitter. 

from scipy.optimize import curve_fit

#p0 is the initial guess for the fitting coefficients (A, mu and sigma above, in that order)
#for more complicated models and fits, the choice of initial conditions is also important 
#to ensuring that the fit will converge. We will see this later.

param_names = ["norm", "mean", "width"]
p0 = [contaminated_pulse.max(), 5., -1]

coeff, cm = curve_fit(gauss, time_axis, contaminated_pulse, p0=p0, sigma=errors)
coeff_error = np.sqrt(np.diag(cm))
flux_fit = gauss(fine_time_axis, *coeff)
for name, param,error in zip(param_names, coeff, coeff_error):
    print("{0:8s}: {1:6.2f} +- {2:6.2f}".format(name, param, error))

fig,axes = plt.subplots(2,1, sharex=True, figsize=[8,6])
axes[0].errorbar(time_axis, contaminated_pulse, errors, fmt='o')
axes[0].plot(fine_time_axis, flux_fit,"--")

residuals = contaminated_pulse - gauss(time_axis, *coeff)
axes[1].errorbar(time_axis, residuals, errors, fmt='o')
axes[1].plot(time_axis, np.zeros(len(time_axis)), '--')

for a in axes:
    a.set_xlabel("t (ns)")
axes[0].set_ylabel("V (mV)")
axes[1].set_ylabel("data - fit")
plt.show()
plt.hist(residuals, bins=20)
plt.xlabel("data-fit")

So it looks like our fit did a poor job, there is still a lot of structure in the residuals, and we can tell by eye that the fit simply didn't converge. 

One method of testing how good a fit is is the $\chi^2$ test. It is basically a measure of the distance the data points are away from the fit. The simplest version (one without error bars) can be calculated as follows:

$$ \chi^2 = \sum_{i} \frac{(y_i - f(x_i))^2}{\sigma_i^2}$$

where $f(x)$ is your model (i.e. the function you are fitting), $x_i$ is the independent variable and $y_i$ and $\sigma_i$ are are your measurements and their uncertainties.

Generally, the number we want to quote is the "chi-square per degrees of freedom" ($\chi^2 / NDF$), which takes into consideration the number of data points going into a fit. If you have more data points than you do paremeters in your model, in general you will produce a better fit. The number of degrees of freedom is calculated as

$$ NDF = k - n_{p} $$

where $n_{p}$ is the number of parameters in your fit. So, if we were fitting a straight line (2 parameters, slope and intercept) to a set of 10 measurements, we would have 8 degrees of freedom.

In [ ]:
def calc_chisquare(meas, sigma, fit):
    
    diff = pow(meas-fit, 2.)
    test_statistic = (diff / pow(sigma,2.)).sum()
    
    return test_statistic
    
TS = calc_chisquare(contaminated_pulse, errors, gauss(time_axis, *coeff))
NDF = len(contaminated_pulse) - len(coeff)
print("chisquare/NDF = {0:.2f} / {1:d}  = {2:.2f}".format(TS, NDF, TS / float(NDF)))

This isn't a great $\chi^2$ -- let's see if we can make it better. 
We expect a good fit to have a $\chi^2 / NDF$ of about 1 -- this means that all data points lie within ~1 error bar of the fit vale.
If a $\chi^2/NDF$ value is too small, it usually means that our errorbars are too large; conversely if the errorbars are too small we can end up with a huge chisquare, so this must all be taken with a bit of a grain of salt.

Firstly, we are fitting a simple Gaussian, which doesn't account for the offset from zero away from the pulse -- this is because we have a **pedestal**, which can be thought of as what is seen by the electronics even when there is no signal. 

One way to handle this is to subtract off the pedestal, but how do we compute it? 

If we make sume assumptions (e.g. our signal arrives somwehre around the middle of our data) then we can calculate the pedestal and subtract it from our data, which should help the fit significantly. 

In [ ]:
slice_condition = time_axis <= 4
pedestal_area = time_axis[slice_condition]

In [ ]:
## What's going on here? 

#We are selecting a view of time_axis where time_axis is <= 4;
#this turns returns a truth table that we can use to slice out relevant data from our signal:
print(slice_condition)
print(pedestal_area)

pedestal_values = contaminated_pulse[slice_condition]
print(pedestal_values)

plt.errorbar(time_axis[slice_condition], contaminated_pulse[slice_condition], errors[slice_condition], fmt='o')
plt.xlabel("t (ns)")
plt.ylabel("V (mV)")
plt.show()

In [ ]:
#Now we can manually compute the mean pedestal, subtract it off of our data, and plot it:
pedestal = pedestal_values.mean()
print("Pedestal: {0:.2f}".format(pedestal))
ped_subtracted_pulse = contaminated_pulse - pedestal

plt.errorbar(time_axis, ped_subtracted_pulse, errors, fmt="o")
plt.xlabel("t (ns)")
plt.ylabel("V (mV) (pedestal-subtracted)")
plt.show()

In [ ]:
param_names = ["norm", "mean", "width"]
p0 = [contaminated_pulse.min(), 7., 1.]

coeff, cm = curve_fit(gauss, time_axis, ped_subtracted_pulse, p0=p0, sigma=errors)
coeff_error = np.sqrt(np.diag(cm))
flux_fit = gauss(time_axis, *coeff)
for name, param,error in zip(param_names, coeff, coeff_error):
    print("{0:8s}: {1:6.2f} +- {2:6.2f}".format(name, param, error))

fig,axes = plt.subplots(2,1, sharex=True, figsize=[8,6])
axes[0].errorbar(time_axis, ped_subtracted_pulse, errors, fmt='.')
axes[0].plot(time_axis, flux_fit,"--")

residuals = ped_subtracted_pulse - flux_fit
axes[1].errorbar(time_axis, residuals, errors, fmt='.')
axes[1].plot(time_axis, np.zeros(len(time_axis)), '--')

for a in axes:
    a.set_xlabel("t (ns)")
axes[0].set_ylabel("V (mV)")
axes[1].set_ylabel("data - fit")

plt.show()
plt.hist(residuals, bins=10)
plt.xlabel("data-fit")
plt.show()

TS = calc_chisquare(ped_subtracted_pulse, errors, gauss(time_axis, *coeff))
NDF = len(ped_subtracted_pulse) - len(coeff)
print("chisquare/NDF = {0:.2f} / {1:d}  = {2:.2f}".format(TS, NDF, TS / float(NDF)))

Excellent! We have a good fit now; there doesn't appear to be anything in the residuals but random noise centered at zero. We could even fit that distribution and determine how well it is fitted by a Gaussian, which would give us a measure of how random our noise is (e.g. the noise could be **correlated** noise, which is very difficult to handle in experiments). 

Alternatively to calculating the pedestal, we can try to include it in our model of the signal. To do this, we will change our fit function to include a parameter which is the pedestal. This is often the way things are done, since you usually don't have *a priori* information about your signal. 

In [ ]:
def gaussian_with_pedestal(x, *p):
    '''
    This is a Gaussian fit with a pedestal. 
    '''
    return gauss(x, *p[0:3]) + p[3]

In [ ]:
param_names = ["norm", "mean", "width", "pedestal"]
p0 = [contaminated_pulse.min(), 6., 1., -10]

coeff, cm = curve_fit(gaussian_with_pedestal, time_axis, contaminated_pulse, p0=p0, sigma=errors)
coeff_error = np.sqrt(np.diag(cm))
flux_fit = gaussian_with_pedestal(time_axis, *coeff)
for name, param,error in zip(param_names, coeff, coeff_error):
    print("{0:8s}: {1:6.2f} +- {2:6.2f}".format(name, param, error))

fig,axes = plt.subplots(2,1, sharex=True, figsize=[8,6])
axes[0].errorbar(time_axis, contaminated_pulse, errors, fmt='.')
axes[0].plot(time_axis, flux_fit,"--")

residuals = contaminated_pulse - flux_fit
axes[1].errorbar(time_axis, residuals, errors, fmt='o')
axes[1].plot(time_axis, np.zeros(len(time_axis)), '--')

for a in axes:
    a.set_xlabel("t (ns)")
axes[0].set_ylabel("V (mV)")
axes[1].set_ylabel("data - fit")

plt.show()
plt.hist(residuals, bins=10)
plt.xlabel("data-fit")
plt.show()

In [ ]:
## We can plot this alongside the ACTUAL gaussian signal we generated and see how closely the fit 
## matches the real thing.

true_data = -gauss(time_axis, amplitude, mean, width) + pedestal

fig,axes = plt.subplots(2,1, sharex=True, figsize=[8,6])
axes[0].errorbar(time_axis, contaminated_pulse, errors, fmt='.')
axes[0].plot(time_axis, flux_fit,"g--", label="Fit")
axes[0].plot(time_axis, true_data, 'r--', label="True")
axes[0].legend()
residuals = contaminated_pulse - flux_fit
axes[1].errorbar(time_axis, residuals, errors, fmt='go')
residuals = contaminated_pulse - true_data
axes[1].errorbar(time_axis, residuals, errors, fmt='ro')
axes[1].plot(time_axis, np.zeros(len(time_axis)), '--')

for a in axes:
    a.set_xlabel("t (ns)")
axes[0].set_ylabel("V (mV)")
axes[1].set_ylabel("data - fit")

plt.show()

plt.hist(contaminated_pulse - flux_fit, bins=20, label="Fit", alpha=0.25)
plt.hist(contaminated_pulse - true_data, bins=20, label="True", alpha=0.25)
plt.legend()
plt.xlabel("data-fit")
plt.show()

In the next example, we have some continum emission and three spectral lines (which we are assuming are Gaussian in shape). 

In-class exercise: We will write our own *simple* fitter which will ignore uncertainties on measurements. The simplest way to do this is to do a least-squares fit. 

We will use the **scipy.optimize** libraries and minimize() function. 

You will want to define a function that you want to minimize, your so-called 'loss function', which is basically just a residuals calculation. You want to minimize your *mean squared* residual. 

Compare your result to the optimize.leastsq() function!

The examples below are **incomplete**; we will likely split this into two different notebooks and discuss getting good parameter estimates to use as our initial guess, since it is very important. 

In [ ]:
def linear(x,*params):
    a,b = params
    return a*x + b

energy = np.linspace(10,100,100)
spectral_line1 = gauss(energy, *[50, 42.1, 3])
spectral_line2 = gauss(energy, *[100, 68.7, 1])
spectral_line3 = gauss(energy, *[100, 18.1, 0.5])
counts = linear(energy,-3,500) + spectral_line1 + spectral_line2 + spectral_line3 + np.random.normal(0, 3, size=len(energy)) 

#for this example, we will say that the errors on our measurements have to do with the counting errors
#So we will use Poissonian statistics here:
errors = np.sqrt(counts)
#Note that this assumes that the response of the detector is uniform across all energies; in reality this is not the case; 
#These would get tied up in **systematic** uncertainties and the instrument response function. 
#We'll touch on this another time.

plt.figure(figsize=[8,6])
plt.errorbar(energy,counts,errors, fmt='.')
plt.xlabel("energy (GeV)")
plt.ylabel("flux (counts)")
plt.show()

In [ ]:
import scipy.optimize as sciopt

In [ ]:
def residuals(parameters, data_x, data_y, model):
    '''
    Returns the residuals of a model vs data.
    '''
    return data_y - model(data_x, *parameters)

## Test using scipy.optimize.leastsq on a dummy fitting case.
data_x = np.array([0,1,2,3,4])
data_y = 5 * data_x + 3 + np.random.normal(0,0.5, size=len(data_x))

model = linear

coeff, _ = sciopt.leastsq(residuals,[0.1, 1], args=(data_x, data_y, model))
plt.plot(data_x,data_y, 'o')
fit_x = np.linspace(data_x[0], data_x[-1])
plt.plot(fit_x, model(fit_x, *coeff), '--')
plt.show()

print(coeff)

In [ ]:
def loss_function(parameters, data_x, data_y, model, verbose=False):
    '''
    This is a really generic loss function. 
    It can take in any number of parameters, any generic model.
    
    Notice that the parameters are passed *first*.
    This is because of the way scipy's libraries need the function to be formatted.
    '''
    
    loss = pow(residuals(parameters, data_x, data_y, model), 2.).sum()
    if verbose:
        print(loss)
    return loss

guess = np.array([50. , 312.])
sciopt.minimize(loss_function, guess, args=(data_x, data_y, model))

In [ ]:
def custom_model(x, *params):
    return linear(x, *params[0:2]) + gauss(x, *params[2:5]) + gauss(x, *params[5:8]) + gauss(x, *params[8:11])


guess = np.array([-10, 500, 200, 20, 3, 200, 40, 3, 200, 68, 3], dtype=float)

fit_energy=np.linspace(energy[0], energy[-1], len(energy)*100)
coeff, cov = sciopt.curve_fit(custom_model, energy, counts, p0=guess, sigma=errors)
coeff_error = np.sqrt(np.diag(cov))
param_names = ["Continuum slope", "continuum intercept", 
               "peak0_A", "peak0_mu", "peak0_sigma", 
               "peak1_A", "peak1_mu", "peak1_sigma",
               "peak2_A", "peak2_mu", "peak2_sigma"]

for name, param,error in zip(param_names, coeff, coeff_error):
    print("{0:30s}: {1:6.2f} +- {2:6.2f}".format(name, param, error))
    
plt.figure(figsize=[8,6])
plt.errorbar(energy,counts,errors, fmt='.')
plt.xlabel("energy (GeV)")
plt.ylabel("flux (counts)")
plt.plot(fit_energy, custom_model(fit_energy, *coeff))
plt.show()

TS = calc_chisquare(counts, errors, custom_model(energy, *coeff))
NDF = len(contaminated_pulse) - len(coeff)
print("chisquare/NDF = {0:.2f} / {1:d}  = {2:.2f}".format(TS, NDF, TS / float(NDF)))

In [ ]:
res = sciopt.minimize(loss_function, guess, args=(energy, counts, custom_model, False))
my_coeff = res.x
plt.errorbar(energy, counts, errors, fmt='.')
plt.plot(energy, custom_model(energy, *my_coeff))
plt.show()


for name, param in zip(param_names, my_coeff):
    print("{0:30s}: {1:6.2f}".format(name, param))
    

TS = calc_chisquare(counts, errors, custom_model(energy, *my_coeff))
NDF = len(counts) - len(my_coeff)
print("chisquare/NDF = {0:.2f} / {1:d}  = {2:.2f}".format(TS, NDF, TS / float(NDF)))    